# **Loading, Exploring, and Converting Dataset to Parquet Format**

This notebook aims to achieve immediate performance improvements throughout the project, especially during the exploratory data analysis (EDA) and feature engineering phases. The primary objective is to convert the dataset from CSV to Parquet format, leveraging the significant benefits it offers in terms of reduced disk space usage and faster data loading. By conducting a comprehensive performance comparison between Parquet and CSV formats, we evaluate their efficiency in terms of read/write speed, disk space utilization, and data compression. Through this analysis, we identify the most suitable format that optimizes the dataset's performance characteristics.

In this notebook, we begin by loading the dataset for the first time and providing an overview of its dimensions. The dataset files in CSV format are located in the `dataset/csv/` directory. Subsequently, a benchmark is performed to determine the optimal persistence method, with the ultimate goal of transitioning from CSV to Parquet format. The converted files in the identified best Parquet format will be stored in the `dataset/pqt/` directory. Additionally, during the benchmarking process, subdirectories such as `dataset/pqt/{engine}_{compression}/` will be created to store different versions of the Parquet files, but these directories will be cleaned up in the final step of the notebook's cleanup process.

**Important Note**:
> The original `HomeCredit_columns_description.csv` file contained a corrupted character at position 59, which caused a `UnicodeDecodeError` when attempting to read it using UTF-8 encoding. If no action is taken, loading the file with UTF-8 encoding results in the following error:
```sh
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 59: invalid start byte
```
> The most practical solution locally is to manually remove and rewrite the character at position 59 using a text editor and save the file. However, in the context of a fully automated process that starts from the raw source without any manual intervention, we opted for loading the file using ISO-8859-1 (Latin-1) encoding. This choice was made after ensuring that there is no loss of information associated with this reduction in the encoding space:
```python
import pandas as pd
csv_dir = "../../dataset/csv/"
filename = "HomeCredit_columns_description.csv"
data = pd.read_csv(csv_dir + filename, encoding="iso-8859-1")
filename_2 = "HomeCredit_columns_description_2.csv"  # manually fixed file
data_2 = pd.read_csv(csv_dir + filename_2, encoding="utf-8")
display(any(data == data_2))
```
> The output will show `True` if the data from the ISO-8859-1 encoded file matches the manually fixed file.

# Reading CSV files

Alright, let's dive into the code! Our goal here is to load each table from the dataset and store them in a convenient dictionary format using dataframes. We'll go through all the CSV files located in a specified directory and measure the time it takes to load each table. The loaded dataframes will be organized within a dictionary, where each table is associated with a unique key. This approach allows us to easily access and manipulate the data. During the loading process, we'll display the table names, their respective shapes, and the time taken to load them. Finally, we'll also calculate and present the total time spent loading all the tables. Let's get started and load those tables!

In [20]:
from pepper.persist import _get_filenames_glob  # Importing function to get CSV filenames
from pepper.utils import pretty_timedelta_str, bold  # Importing utility functions
import pandas as pd  # Importing pandas for data manipulation
import time  # Importing time for measuring execution time

csv_dir = "../../dataset/csv/"  # Directory path where CSV files are located
filenames = _get_filenames_glob(csv_dir, "csv")  # Get list of CSV filenames

data_dict = {}  # Dictionary to store dataframes
read_times = []  # List to store loading times of each table

for filename in filenames:
    t = -time.time()  # Start measuring loading time
    data_key = filename[:-4]  # Extract data key from filename
    # data = pd.read_csv(csv_dir + filename, encoding="utf-8")
    # Read CSV file into a dataframe
    try:  # Try reading with UTF-8 encoding
        data = pd.read_csv(csv_dir + filename, encoding="utf-8")
    except UnicodeDecodeError:  # Fallback to ISO-8859-1 encoding
        data = pd.read_csv(csv_dir + filename, encoding="iso-8859-1")
    t += time.time()  # Calculate elapsed time for loading
    read_times.append(t)  # Store loading time in the list
    data_dict[data_key] = data  # Add dataframe to the dictionary
    # Display table name, shape, and loading time
    print(f"{bold(data_key)}: {data.shape} - {pretty_timedelta_str(t, 2)}")

# Display total read time for all tables
print(f">> {bold('total read time')}: {pretty_timedelta_str(sum(read_times), 2)}")

application_test: (48744, 121) - 658 ms, 555 mus
application_train: (307511, 122) - 3 s, 547 ms
bureau: (1716428, 17) - 2 s, 771 ms
bureau_balance: (27299925, 3) - 5 s, 941 ms
credit_card_balance: (3840312, 23) - 9 s, 555 ms
HomeCredit_columns_description: (219, 5) - 11 ms, 86 mus
installments_payments: (13605401, 8) - 13 s, 606 ms
POS_CASH_balance: (10001358, 8) - 8 s, 219 ms
previous_application: (1670214, 37) - 8 s, 321 ms
sample_submission: (48744, 2) - 14 ms, 486 mus
>> total read time: 52 s, 647 ms


# Dataset Overview

In this section, we create a `metadata` dataframe that provides an overview of the dimensions of our dataset, which consists of multiple tables. This information can be useful for gaining insights into the structure and size of the dataset. We calculate the number of samples, number of features, and the total number of cells in each table. Additionally, we retrieve the size of each corresponding CSV file and add the CSV read times to the `metadata`. Finally, we sort the `metadata` dataframe by the number of cells in descending order and display the results.

In [5]:
# Import necessary modules
from pepper.utils import bold
from pepper.utils import get_file_size
import pandas as pd

# Create metadata dataframe to provide an overview of the dataset dimensions
metadata = pd.DataFrame(
    [(key, *data.shape) for key, data in data_dict.items()],
    columns=["table_name", "n_samples", "n_features"]
)

# Calculate the total number of cells in each table
metadata["n_cells"] = metadata.n_samples * metadata.n_features

# Retrieve the size of each CSV file
metadata["csv_size"] = metadata.table_name.apply(
    lambda x: get_file_size(csv_dir + x + ".csv")
)

# Add CSV read times to the metadata dataframe
metadata["csv_read_time"] = read_times

# Sort the metadata dataframe by the number of cells in descending order
metadata = metadata.sort_values(by="n_cells", ascending=False)

# Print the total number of cells in the dataset using bold formatting
print(f"{bold('n_cells')}: {metadata.n_cells.sum():n}")

# Display the metadata dataframe
display(metadata)

n_cells: 493 571 166


,table_name,n_samples,n_features,n_cells,csv_size,csv_read_time
6,installments_payments,13605401,8,108843208,723118349,24.694108
4,credit_card_balance,3840312,23,88327176,424582605,14.245052
3,bureau_balance,27299925,3,81899775,375592889,7.529640
7,POS_CASH_balance,10001358,8,80010864,392703158,15.538277
8,previous_application,1670214,37,61797918,404973293,16.084218
1,application_train,307511,122,37516342,166133370,6.033286
2,bureau,1716428,17,29179276,170016717,4.938051
0,application_test,48744,121,5898024,26567651,1.050658
9,sample_submission,48744,2,97488,536202,0.044487
5,HomeCredit_columns_description,219,5,1095,37391,0.015399


# Benchmark: Saving Dataset to Parquet with Various Configurations

When saving the dataset in the Parquet format, the default configuration uses the `pyarrow` engine for processing. If `pyarrow` is not available, it falls back to using the `fastparquet` engine. The data is compressed using the `snappy` compression algorithm.

Our goal is to compare the performance in terms of disk memory usage and loading time for six possible configurations. These configurations depend on the choice of engine (`pyarrow` or `fastparquet`) and compression (`snappy`, `gzip`, `brotli`, or no compression).

For each configuration, we create a dedicated subdirectory to store the Parquet files. The dataset is then saved to Parquet using the specified engine and compression. The progress of the operation is displayed, indicating the configuration and the time taken.

Please note that we have excluded the combination of `fastparquet` engine with `brotli` compression due to an unidentified issue that causes the execution to enter an infinite loop.

**Warning**: If you plan to rerun this benchmark, please allocate approximately 10 minutes for the execution.

**Further readings**:
* [**`pandas.DataFrame.to_parquet`** documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html)
* [**Stack Overflow**: Python - Save pandas data frame to Parquet file](https://stackoverflow.com/questions/41066582/python-save-pandas-data-frame-to-parquet-file)

In [6]:
# Import necessary modules
from pepper.persist import all_to_parquet
import itertools
import time

# Define CSV and Parquet directories
csv_dir = "../../dataset/csv/"
pqt_dir = "../../dataset/pqt/"

# Define the list of engines and compressions to test
engines = ["pyarrow", "fastparquet"]
compressions = ["snappy", "gzip", "brotli", None]

# Iterate over all combinations of engines and compressions
for engine, compression in itertools.product(engines, compressions):
    # Skip the combination of `fastparquet` and `brotli` due to an unidentified issue
    # Note: Saving `application_train` does not crash but remains stuck indefinitely
    if engine == "fastparquet" and compression == "brotli":
        continue
    
    # Create a configuration name based on the engine and compression
    config_name = f"{engine}_{str(compression).lower()}"
    
    # Define the subdirectory for the Parquet files
    pqt_subdir = pqt_dir + config_name + "/"
    
    # Measure the time taken to save the dataset to Parquet
    t = -time.time()
    print(f"Saving dataset to {pqt_subdir}", end="")
    all_to_parquet(data_dict, pqt_subdir, engine, compression)
    t += time.time()
    print(f" in {pretty_timedelta_str(t, 2)}")

Saving dataset to ../../dataset/pqt/pyarrow_snappy/.......... in 54 s, 348 ms
Saving dataset to ../../dataset/pqt/pyarrow_gzip/.......... in 2 m, 51 s
Saving dataset to ../../dataset/pqt/pyarrow_brotli/.......... in 3 m, 36 s
Saving dataset to ../../dataset/pqt/pyarrow_none/.......... in 43 s, 289 ms
Saving dataset to ../../dataset/pqt/fastparquet_snappy/.......... in 48 s, 329 ms
Saving dataset to ../../dataset/pqt/fastparquet_gzip/.......... in 4 m, 43 s
Saving dataset to ../../dataset/pqt/fastparquet_none/.......... in 1 m, 56 s


# Read Comparison: Parquet vs CSV Formats

The objective here is to compare the performance of the different configurations (engines and compressions) when reading Parquet files.

For each combination of _engine_ and _compression_, we measure the _file size_ and _read time_ for each table in the `metadata` dataframe. The measurements are stored in the `metadata` dataframe for further analysis and comparison.

In [7]:
from pepper.utils import get_file_size

# Function to measure the time taken to read a Parquet file
def pqt_read_time(pqt_dir, table_name):
    t = time.time()
    pd.read_parquet(pqt_dir + table_name + ".pqt")
    return time.time() - t

# Function to get the file size of a Parquet file
def pqt_file_size(pqt_dir, table_name):
    return get_file_size(pqt_dir + table_name + ".pqt")

# List of engines and compressions to iterate over
engines = ["pyarrow", "fastparquet"]
compressions = ["snappy", "gzip", "brotli", None]

# Iterate over all combinations of engines and compressions
for engine, compression in itertools.product(engines, compressions):
    # Skip the combination of `fastparquet` and `brotli` due to an unidentified issue
    # Note: Saving `application_train` does not crash but remains stuck indefinitely
    if engine == "fastparquet" and compression == "brotli":
        continue
    config_name = f"{engine}_{str(compression).lower()}"
    pqt_subdir = pqt_dir + config_name + "/"
    
    # Calculate and store the file size for each table using the specified engine and compression
    metadata[f"pqt_{config_name}_size"] = metadata.table_name.apply(
        lambda x: pqt_file_size(pqt_subdir, x)
    )
    
    # Measure and store the read time for each table using the specified engine and compression
    metadata[f"pqt_{config_name}_read_time"] = metadata.table_name.apply(
        lambda x: pqt_read_time(pqt_subdir, x)
    )

# Display the updated metadata dataframe
display(metadata)

,table_name,n_samples,n_features,n_cells,csv_size,csv_read_time,pqt_pyarrow_snappy_size,pqt_pyarrow_snappy_read_time,pqt_pyarrow_gzip_size,pqt_pyarrow_gzip_read_time,pqt_pyarrow_brotli_size,pqt_pyarrow_brotli_read_time,pqt_pyarrow_none_size,pqt_pyarrow_none_read_time,pqt_fastparquet_snappy_size,pqt_fastparquet_snappy_read_time,pqt_fastparquet_gzip_size,pqt_fastparquet_gzip_read_time,pqt_fastparquet_none_size,pqt_fastparquet_none_read_time
6,installments_payments,13605401,8,108843208,723118349,24.694108,330470104,30.688298,246648550,7.286146,234206927,4.913854,478259694,4.746378,417551342,4.922243,273744883,6.731994,874103290,24.645302
4,credit_card_balance,3840312,23,88327176,424582605,14.245052,111274155,4.277819,87382864,3.215717,84062930,3.428892,231893301,2.734187,158525573,2.396325,99998309,3.780858,671997342,13.958750
3,bureau_balance,27299925,3,81899775,375592889,7.529640,21426895,3.449970,7220080,3.604056,6528751,3.906244,212427359,6.685958,39104894,2.520526,8773070,2.696896,573299674,12.917735
7,POS_CASH_balance,10001358,8,80010864,392703158,15.538277,124435906,4.470653,89478858,2.158790,84330013,2.670905,192425645,3.399509,166379319,2.772187,93648196,3.964802,664506876,15.570412
8,previous_application,1670214,37,61797918,404973293,16.084218,62912447,4.510714,49908242,3.500851,48304590,3.388146,80714342,2.748807,115293797,2.974745,62131222,3.793459,514893753,12.225278
1,application_train,307511,122,37516342,166133370,6.033286,22225869,1.368119,18770994,0.845013,18486431,0.804159,24879919,0.883408,49802974,1.026025,25306399,1.379967,253550609,10.481033
2,bureau,1716428,17,29179276,170016717,4.938051,35241265,1.553160,25883443,0.924298,24365040,0.849103,61232824,0.976085,52235506,1.546416,29284991,1.412190,234062490,6.331937
0,application_test,48744,121,5898024,26567651,1.050658,4255523,0.255053,3596498,0.256882,3505436,0.135322,4861820,0.265691,8361289,0.341156,4258899,0.722261,40157544,1.139118
9,sample_submission,48744,2,97488,536202,0.044487,296358,0.064025,170995,0.068112,156444,0.021795,489947,0.047901,215625,0.104771,77364,0.075130,780849,0.071879
5,HomeCredit_columns_description,219,5,1095,37391,0.015399,13372,0.031034,10505,0.029900,9931,0.033183,23605,0.017045,10992,0.045842,7179,0.131685,41639,0.064639


Based on the summary below, the best choice seems to be `pyarrow` + `gzip`.

This is the default configuration that we have settled on.

It provides a 5x improvement in both speed and disk footprint.

In [8]:
# Display the sum of each column in the `metadata` DataFrame
display(metadata.sum(axis=0))

table_name                          installments_paymentscredit_card_balancebureau...
n_samples                                                                    58538856
n_features                                                                        346
n_cells                                                                     493571166
csv_size                                                                   2684261625
csv_read_time                                                               90.173177
pqt_pyarrow_snappy_size                                                     712551894
pqt_pyarrow_snappy_read_time                                                50.668844
pqt_pyarrow_gzip_size                                                       529071029
pqt_pyarrow_gzip_read_time                                                  21.889765
pqt_pyarrow_brotli_size                                                     503956493
pqt_pyarrow_brotli_read_time                          

Let's generate a _pretty_ table to present these results on presentation slides (the Markdown table can be copied to the clipboard).

The code snippet below takes the `metadata` DataFrame, selects the desired columns for the resulting table, and creates a copy of it. It then calculates the total values for each column and assigns them to the "TOTAL" row. Next, it applies formatting functions to specific columns to format file sizes, time durations, and large integers. The column names are modified for better readability. Finally, the formatted table is displayed using the `display_dataframe_in_markdown` function.

In [9]:
from pepper.utils import display_dataframe_in_markdown, format_iB, pretty_timedelta_str

# Function to format file size
def format_size(x):
    sz, unity = format_iB(x)
    return f"{sz:.1f} {unity}"

# Function to format time duration
def format_time(x):
    return pretty_timedelta_str(x, 1)

# Function to format large integers with thousand separators
def format_bigint(x):
    return f"{x:n}"

# Get the columns from the metadata DataFrame
cols = metadata.columns

# Select the desired columns for the resulting table
res_cols = list(cols[:6]) + list(cols[cols.str.contains("pyarrow_gzip")])
res = metadata[res_cols]

# Create a copy of the resulting table
res_2 = res.copy()

# Calculate the total values for each column and assign it to the "TOTAL" row
total = res_2.sum(axis=0)
total[0] = "**TOTAL**"
res_2.loc["TOTAL"] = total

# Apply formatting functions to specific columns
res_2.csv_size = res_2.csv_size.apply(format_size)
res_2.pqt_pyarrow_gzip_size = res_2.pqt_pyarrow_gzip_size.apply(format_size)
res_2.csv_read_time = res_2.csv_read_time.apply(format_time)
res_2.pqt_pyarrow_gzip_read_time = res_2.pqt_pyarrow_gzip_read_time.apply(format_time)
res_2.n_samples = res_2.n_samples.apply(format_bigint)
res_2.n_cells = res_2.n_cells.apply(format_bigint)

# Modify column names for better readability
res_2.columns = (
    res_2.columns
    .str.replace("n_", "#")
    .str.replace("pqt_pyarrow_gzip", "parquet")
    .str.replace("read_time", "readtime")
    .str.replace("_", " ")
)

# Format the "TOTAL" row to be displayed in bold
res_2.loc["TOTAL"] = res_2.loc["TOTAL"].apply(lambda x: f"**{x}**")

# Display the formatted table in Markdown format
display_dataframe_in_markdown(res_2)

|table name|#samples|#features|#cells|csv size|csv readtime|parquet size|parquet readtime|
|-|-|-|-|-|-|-|-|
|installments_payments|13 605 401|8|108 843 208|689.6 MiB|24 s|235.2 MiB|7 s|
|credit_card_balance|3 840 312|23|88 327 176|404.9 MiB|14 s|83.3 MiB|3 s|
|bureau_balance|27 299 925|3|81 899 775|358.2 MiB|7 s|6.9 MiB|3 s|
|POS_CASH_balance|10 001 358|8|80 010 864|374.5 MiB|15 s|85.3 MiB|2 s|
|previous_application|1 670 214|37|61 797 918|386.2 MiB|16 s|47.6 MiB|3 s|
|application_train|307 511|122|37 516 342|158.4 MiB|6 s|17.9 MiB|845 ms|
|bureau|1 716 428|17|29 179 276|162.1 MiB|4 s|24.7 MiB|924 ms|
|application_test|48 744|121|5 898 024|25.3 MiB|1 s|3.4 MiB|256 ms|
|sample_submission|48 744|2|97 488|523.6 KiB|44 ms|167.0 KiB|68 ms|
|HomeCredit_columns_description|219|5|1 095|36.5 KiB|15 ms|10.3 KiB|29 ms|
|****TOTAL****|**58 538 856**|**346**|**493 571 166**|**2.5 GiB**|**1 m**|**504.6 MiB**|**21 s**|

# Cleanup

Let's retrieve the files generated in the `pyarrow` + `gzip` configuration and move them to the parent directory `dataset/pqt/` for safekeeping:

In [10]:
from pathlib import Path
import os

def move_files_to_parent(dir_path):
    """Moves all files in the specified directory to its parent directory.
    
    Parameters
    ----------
    dir_path : str
        The path to the directory containing the files.
    """
    parent_dir_path = Path(dir_path).parent
    for file in Path(dir_path).iterdir():
        print(file.name)
        file.rename(parent_dir_path.joinpath(file.name))

pqt_subdir = os.path.join(pqt_dir, "pyarrow_gzip/")
move_files_to_parent(pqt_subdir)

application_test.pqt
application_train.pqt
bureau.pqt
bureau_balance.pqt
credit_card_balance.pqt
HomeCredit_columns_description.pqt
installments_payments.pqt
POS_CASH_balance.pqt
previous_application.pqt
sample_submission.pqt


And let's free up disk space by removing the files in the other parquet formats generated during our benchmark (because life is all about taking risks, right?):

In [11]:
import os, shutil
def _dangerous_rmtree_all_subdirs(dir_path):
    """Recursively removes all subdirectories and their contents within the specified directory.
    
    Parameters
    ----------
    dir_path : str
        The path to the directory to be removed.
    """
    for child_name in os.listdir(dir_path):
        child_path = os.path.join(dir_path, child_name)
        if os.path.isdir(child_path):
            shutil.rmtree(child_path)

_dangerous_rmtree_all_subdirs(pqt_dir)

# Execution Context

Execution context information including the date and time, package versions, system information, and environment variables.

In [22]:
# Import the necessary packages
import datetime
import platform
import pkg_resources
import os

def print_execution_context():
    """Prints the execution context information including date and time,
    package versions, system information, and environment variables.
    """
    # Get the current date and time
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Get the package versions
    package_versions = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # Get the system information
    system_info = f"{platform.system()} {platform.release()}"

    # Get the environment variables
    environment_variables = dict(os.environ)

    # Print the execution context information
    print("Execution Context")
    print("-----------------")
    print("Package Versions:")
    for package, version in package_versions.items():
        print(f"  - {package}: {version}")
    print(f"System Information: {system_info}")
    print("Environment Variables:")
    for key, value in environment_variables.items():
        print(f"  - {key}: {value}")

print_execution_context()

Execution Context
-----------------
Package Versions:
  - pygments: 2.14.0
  - asttokens: 2.2.1
  - backcall: 0.2.0
  - colorama: 0.4.6
  - comm: 0.1.3
  - debugpy: 1.6.6
  - decorator: 5.1.1
  - executing: 1.2.0
  - ipykernel: 6.22.0
  - ipython: 8.12.0
  - jedi: 0.18.2
  - jupyter-client: 8.1.0
  - jupyter-core: 5.3.0
  - matplotlib-inline: 0.1.6
  - nest-asyncio: 1.5.6
  - packaging: 23.0
  - parso: 0.8.3
  - pickleshare: 0.7.5
  - platformdirs: 3.2.0
  - prompt-toolkit: 3.0.38
  - psutil: 5.9.4
  - pure-eval: 0.2.2
  - python-dateutil: 2.8.2
  - pywin32: 306
  - pyzmq: 25.0.2
  - six: 1.16.0
  - stack-data: 0.6.2
  - tornado: 6.2
  - traitlets: 5.9.0
  - wcwidth: 0.2.6
  - babel: 2.12.1
  - flask: 2.2.3
  - heapdict: 1.0.1
  - jinja2: 3.1.2
  - mako: 1.2.4
  - markdown: 3.4.3
  - markupsafe: 2.1.2
  - pillow: 9.5.0
  - pyyaml: 6.0
  - sqlalchemy: 2.0.15
  - send2trash: 1.8.0
  - werkzeug: 2.2.3
  - absl-py: 1.4.0
  - alabaster: 0.7.13
  - alembic: 1.11.1
  - ansi2html: 1.8.0
  - an